<div style="text-align:center">
    <h2 style="display:inline">Pre-processing VDJdb<span class="tocSkip"></span></h2>
</div>

### 1. Load the packages and data

In [1]:
import pandas as pd
from IPython.display import display

In [2]:
rawdata = pd.read_excel('vdjdb.xlsx')

In [3]:
rawdata.head(2)

complex.id gene                  cdr3       v.segm      j.segm  \
0           1  TRA         CIVRAPGRADMRF  TRAV26-1*01   TRAJ43*01   
1           1  TRB  CASSYLPGQGDHYSNQPQHF    TRBV13*01  TRBJ1-5*01   

       species     mhc.a mhc.b mhc.class antigen.epitope  ... antigen.species  \
0  HomoSapiens  HLA-B*08   B2M      MHCI        FLKEKGGL  ...           HIV-1   
1  HomoSapiens  HLA-B*08   B2M      MHCI        FLKEKGGL  ...           HIV-1   

    reference.id                                             method  \
0  PMID:15596521  {"frequency": "", "identification": "tetramer-...   
1  PMID:15596521  {"frequency": "", "identification": "tetramer-...   

                                                meta  \
0  {"cell.subset": "CD8+", "clone.id": "", "donor...   
1  {"cell.subset": "CD8+", "clone.id": "", "donor...   

                                             cdr3fix vdjdb.score  web.method  \
0  {"cdr3": "CIVRAPGRADMRF", "cdr3_old": "CIVRAPG...           2        sort   
1  {"cdr3": "CASSYLPGQGDHYSNQPQHF", "cdr3_old": "...           2        sort   

  web.method.seq web.cdr3fix.nc web.cdr3fix.unmp  
0         sanger             no               no  
1         sanger             no               no  

[2 rows x 21 columns]

In [4]:
rawdata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 92771 entries, 0 to 92770
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   complex.id        92771 non-null  int64 
 1   gene              92771 non-null  object
 2   cdr3              92771 non-null  object
 3   v.segm            92670 non-null  object
 4   j.segm            91626 non-null  object
 5   species           92771 non-null  object
 6   mhc.a             92771 non-null  object
 7   mhc.b             92771 non-null  object
 8   mhc.class         92771 non-null  object
 9   antigen.epitope   92771 non-null  object
 10  antigen.gene      92709 non-null  object
 11  antigen.species   92771 non-null  object
 12  reference.id      91260 non-null  object
 13  method            92771 non-null  object
 14  meta              92771 non-null  object
 15  cdr3fix           92771 non-null  object
 16  vdjdb.score       92771 non-null  int64 
 17  web.method  

In [5]:
# dtype_df = pd.DataFrame(columns=['Column', 'Data Type'])

# for column in rawdata.columns:
#     data_type = type(rawdata[column][0]).__name__
#     dtype_df = pd.concat([dtype_df, pd.DataFrame({'Column': [column], 'Data Type': [data_type]})], ignore_index=True)

# print(dtype_df)

### 2. Data describtion

From the data type of each column we can see that all are of type **str** except *conplex.id* and *vdjdb.score* which are of type **int**.

The data contains a total of 21 columns, we can make an initial filter by the type and meaning of each column.

In [6]:
rawdata_info = pd.read_excel('vdjdb.xlsx', sheet_name='Sheet2')

In [7]:
def display_left_aligned(df):
    style = df.style.set_properties(**{
        'text-align': 'left',
        'white-space': 'pre-wrap',
    })

    style.set_table_styles([{
        'selector': 'th',
        'props': [('text-align', 'left')]
    }])
    
    display(style)

In [8]:
display_left_aligned(rawdata_info)

### 3. Data wrangling

**Removing columns**

Some of the columns contain specific information about the literature sources (reference.id), sequencing methods (method, web.method.seq), collection method (web.method) etc., some of which are related to how **vdjdb.score** is calculated.

Therefore we first remove these columns. Deleted columns are as follows: 
-  reference.id
-  method
-  meta
-  cdr3fix
-  web.method  
-  web.method.seq
-  web.cdr3fix.nc
-  web.cdr3fix.unmp

In [9]:
data = rawdata.iloc[:, list(range(0, 12)) + [16]]
data.head(2)

complex.id gene                  cdr3       v.segm      j.segm  \
0           1  TRA         CIVRAPGRADMRF  TRAV26-1*01   TRAJ43*01   
1           1  TRB  CASSYLPGQGDHYSNQPQHF    TRBV13*01  TRBJ1-5*01   

       species     mhc.a mhc.b mhc.class antigen.epitope antigen.gene  \
0  HomoSapiens  HLA-B*08   B2M      MHCI        FLKEKGGL          Nef   
1  HomoSapiens  HLA-B*08   B2M      MHCI        FLKEKGGL          Nef   

  antigen.species  vdjdb.score  
0           HIV-1            2  
1           HIV-1            2

**Null values**

The colums v.segm and j.segm have 101, 1145 null values respectively. Due to the fact 

In [10]:
data.isnull().sum()

complex.id            0
gene                  0
cdr3                  0
v.segm              101
j.segm             1145
species               0
mhc.a                 0
mhc.b                 0
mhc.class             0
antigen.epitope       0
antigen.gene         62
antigen.species       0
vdjdb.score           0
dtype: int64

**complex.id**

Due to the fact that TCR alpha and beta chain records belonging to the same T-cell clone, and in case the complex.id is equal to zero, a record doesn't have either alpha or beta chain, we count both zero and non-zero values, and then check if the non-zero values occur twice.

In [13]:
zeros = data[data['complex.id'] == 0]
zeros_count = zeros['complex.id'].count()
zeros_a_count = zeros[zeros['gene'] == 'TRA']['gene'].count()
zeros_b_count = zeros[zeros['gene'] == 'TRB']['gene'].count()
print(f"The column complex.id contains a total of {zeros_count} values that are equal to 0.\n"
      f"Among them, there are {zeros_a_count} belonging to the alpha chain"\
      f"and {zeros_b_count} belonging to the beta chain.")

non_zeros = data[data['complex.id'] != 0]['complex.id']
non_zeros_counts = non_zeros.value_counts()

# Check if all non-zero values occur twice
pairs_check = all(non_zeros_counts == 0)

if pairs_check:
    print("All non-zero values occur in pairs except for 0.")
else:
    print("There are non-paired values among non-zero values.")

The column complex.id contains a total of 31583 values that are equal to 0.
Among them, there are 7455 belonging to the alpha chainand 24128 belonging to the beta chain.
There are non-paired values among non-zero values.


### 4. Inference

Based on various measures we could use to calculate the distance/similarity matrix, the columns we choose as the input are different. Besides we should calculate these metrics for the alpha and beta chains seperately in the further study.

**Therefore we don't have to remove the lines with null values and complex.id values that are equal to 0 right away.**

##### TCRdist3

Tcrdist3 only requires 3 input columns for single chain analysis (i.e., for beta chain cdr3_b_aa and v_b_gene, j_b_gene) and 6 columns for paired chain analysis (i.e., cdr3_b_aa, v_b_gene, j_b_gene, cdr3_a_aa, v_a_gene, and j_a_gene).

A unique feature of tcrdist3 is that all of the parameters of the distance metric can be adjusted (e.g. alpha-chain only, weights on CDR loops, etc.) or completely new user-defined metrics can be provided to calculate pairwise distances. The package comes with a distance based on Needleman-Wunsch global sequence alignment and a BLOSUM62 similarity matrix, as well as the Levenshtein/edit distance, which is employed by other TCR analysis packages such as *TCRNET/VDJtools*, *ALICE*, and *GLIPH2*.

https://tcrdist3.readthedocs.io/en/latest/index.html

##### GLIPH

Input TCR data are formated as follow
#CDR3b  TRBV  TRBJ  CDR3a  subject:condition count
CSARDQGGAGNQPQHF	TRBV20-1	TRBJ1-5	CAVGVGYKLSF	01/0906:MtbLys	1

All fields are tab-delimited except that subject and condition are delimited with ":". Condition can be anything such as tissue type, cell subset or treatment et al. CDR3b, TRBV, subject, and count are required. Other fields can be replaced with "NA". A demo input TCR dataset can be found at the link [TCR](http://50.255.35.37:8080/demo).

Huang, Huang, et al. "Analyzing the Mycobacterium tuberculosis immune response by T-cell receptor clustering with GLIPH2 and genome-wide antigen screening." Nature Biotechnology 38.10 (2020): 1194-1202. [Link](https://www.nature.com/articles/s41587-020-0505-4)

GLIPH1 works well on small and clean data sets. However, as data sets are becoming larger and noisier, the algorithm tends to generate large clusters of mixed specificities. [Link](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC10035763/).

http://50.255.35.37:8080/tools

https://github.com/immunoengineer/gliph

##### GIANA

Input of GIANA is flexible. The first column is kept for CDR3 amino acid sequence. If TRBV allele information is enabled (by default), the second column is required to be TRBV genes. As the TCR-seq data provided by the Adaptive Biotechnologies does not comply with the IMGT format, we provide the R code (ProcessAdaptiveFile.R) to convert the Adaptive data input to standard format. In the output, GIANA inserts a column between the first and the second column as the cluster IDs. Other columns in the input data may contain any information, and will be kept in the final output.

https://github.com/s175573/GIANA